# Creating and Valuing a CDS Contract vs Markit

Example CDS Valuation and comparison with market standard ISDA model on Markit website

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from financepy.utils import *
from financepy.products.rates import *
from financepy.products.credit import *

## Creating a CDS Contract

This based on an example from Markit's CDS calculator website https://www.markit.com/markit.jsp?jsppage=pv.jsp

In [3]:
value_dt = Date(21, 11, 2019)
settle_dt = value_dt.add_days(1)
maturity_dt = Date(20, 12, 2024)
cds_cpn = 0.050
notional = ONE_MILLION
long_protection = True
trade_dt = Date(9, 8, 2019)

In [4]:
cds_contract = CDS(settle_dt, maturity_dt, cds_cpn, notional, long_protection)

In [5]:
print(cds_contract)

OBJECT TYPE: CDS
STEP-IN DATE: 22-NOV-2019
MATURITY: 20-DEC-2024
NOTIONAL: 1000000
RUN COUPON: 500.0bp
DAYCOUNT: DayCountTypes.ACT_360
FREQUENCY: FrequencyTypes.QUARTERLY
CALENDAR: CalendarTypes.WEEKEND
BUSDAYRULE: BusDayAdjustTypes.FOLLOWING
DATEGENRULE: DateGenRuleTypes.BACKWARD
ACCRUED DAYS: 63.0
PAYMENT_dt, YEAR_FRAC, ACCRUAL_START, ACCRUAL_END, FLOW
20-DEC-2019,     0.252778, 20-SEP-2019, 19-DEC-2019, 12638.888889
20-MAR-2020,     0.252778, 20-DEC-2019, 19-MAR-2020, 12638.888889
22-JUN-2020,     0.261111, 20-MAR-2020, 21-JUN-2020, 13055.555556
21-SEP-2020,     0.252778, 22-JUN-2020, 20-SEP-2020, 12638.888889
21-DEC-2020,     0.252778, 21-SEP-2020, 20-DEC-2020, 12638.888889
22-MAR-2021,     0.252778, 21-DEC-2020, 21-MAR-2021, 12638.888889
21-JUN-2021,     0.252778, 22-MAR-2021, 20-JUN-2021, 12638.888889
20-SEP-2021,     0.252778, 21-JUN-2021, 19-SEP-2021, 12638.888889
20-DEC-2021,     0.252778, 20-SEP-2021, 19-DEC-2021, 12638.888889
21-MAR-2022,     0.252778, 20-DEC-2021, 20-MAR-20

## Build Ibor Curve

In [6]:
dc_type = DayCountTypes.ACT_360
depo1 = IborDeposit(settle_dt, "1M", 0.017156, dc_type)
depo2 = IborDeposit(settle_dt, "2M", 0.018335, dc_type)
depo3 = IborDeposit(settle_dt, "3M", 0.018988, dc_type)
depo4 = IborDeposit(settle_dt, "6M", 0.018911, dc_type)
depo5 = IborDeposit(settle_dt, "12M", 0.019093, dc_type)
depos = [depo1,depo2,depo3,depo4,depo5]

swap_type = SwapTypes.PAY
dc_type = DayCountTypes.THIRTY_E_360_ISDA
fixed_freq = FrequencyTypes.SEMI_ANNUAL
swap1 = IborSwap(settle_dt,"2Y",swap_type,0.015630,fixed_freq,dc_type)
swap2 = IborSwap(settle_dt,"3Y",swap_type,0.015140,fixed_freq,dc_type)
swap3 = IborSwap(settle_dt,"4Y",swap_type,0.015065,fixed_freq,dc_type)
swap4 = IborSwap(settle_dt,"5Y",swap_type,0.015140,fixed_freq,dc_type)
swap5 = IborSwap(settle_dt,"6Y",swap_type,0.015270,fixed_freq,dc_type)
swap6 = IborSwap(settle_dt,"7Y",swap_type,0.015470,fixed_freq,dc_type)
swap7 = IborSwap(settle_dt,"8Y",swap_type,0.015720,fixed_freq,dc_type)
swap8 = IborSwap(settle_dt,"9Y",swap_type,0.016000,fixed_freq,dc_type)
swap9 = IborSwap(settle_dt,"10Y",swap_type,0.016285,fixed_freq,dc_type)
swap10 = IborSwap(settle_dt,"12Y",swap_type,0.01670,fixed_freq,dc_type)
swaps = [swap1,swap2,swap3,swap4,swap5,swap6,swap7,swap8,swap9,swap10]

libor_curve = IborSingleCurve(value_dt, depos, [], swaps)

# Build a CDS Curve

In [7]:
cdsFlatSpread = 0.0100

In [8]:
cds1 = CDS(settle_dt, "1Y", cdsFlatSpread)
cds2 = CDS(settle_dt, "2Y", cdsFlatSpread)
cds3 = CDS(settle_dt, "3Y", cdsFlatSpread)
cds4 = CDS(settle_dt, "4Y", cdsFlatSpread)
cds5 = CDS(settle_dt, "5Y", cdsFlatSpread)
cds6 = CDS(settle_dt, "7Y", cdsFlatSpread)
cds7 = CDS(settle_dt, "10Y", cdsFlatSpread)
cds8 = CDS(settle_dt, "15Y", cdsFlatSpread)

In [9]:
cdss = [cds1, cds2, cds3, cds4, cds5, cds6, cds7, cds8]

In [10]:
recovery_rate = 0.40

In [11]:
issuer_curve = CDSCurve(value_dt, cdss, libor_curve, recovery_rate)

# Valuation Results

In [12]:
spd = cds_contract.par_spread(value_dt, issuer_curve, recovery_rate) * 10000.0
print("FAIR CDS SPREAD %10.5f bp"% spd)

FAIR CDS SPREAD  100.00000 bp


In [13]:
v = cds_contract.value(value_dt, issuer_curve, recovery_rate)
dirty_pv = v['dirty_pv']
clean_pv = v['clean_pv']

print("DIRTY VALUE %12.2f"% dirty_pv)
print("CLEAN VALUE %12.2f"% clean_pv)

DIRTY VALUE   -198527.95
CLEAN VALUE   -189777.95


MARKIT CALCULATE -198,551 for the DIRTY VALUE

In [14]:
cleanp = cds_contract.clean_price(settle_dt, issuer_curve, recovery_rate)
print("CLEAN PRICE %12.6f"% cleanp)

CLEAN PRICE   118.979525


MARKIT CALCULATE 118.98%

In [15]:
accrued_days = cds_contract.accrued_days()
print("ACCRUED_DAYS", accrued_days)

ACCRUED_DAYS 63.0


MARKIT CALCULATE 63

In [16]:
accrued_interest = cds_contract.accrued_interest()
print("ACCRUED_COUPON", accrued_interest)

ACCRUED_COUPON -8750.0


MARKIT CALCULATE 8750

In [17]:
prot_pv = cds_contract.prot_leg_pv(settle_dt, issuer_curve, recovery_rate)
print("prot_PV", prot_pv)

prot_PV 47448.7007624235


In [18]:
premPV = cds_contract.premium_leg_pv(settle_dt, issuer_curve, recovery_rate)
print("PREMIUM_PV", premPV)

PREMIUM_PV 245993.94714388327


## Risk Measures

In [19]:
credit_dv01 = cds_contract.credit_dv01(settle_dt, issuer_curve, recovery_rate)
print("Credit DV01  %12.2f"% credit_dv01)

Credit DV01        553.56


MARKIT FOUND 554

In [20]:
interest_dv01 = cds_contract.interest_dv01(settle_dt, issuer_curve, recovery_rate)
print("Interest DV01  %12.2f"% interest_dv01)

Interest DV01         48.85


MARKIT FOUND 49

## Full Analysis 

In [21]:
cds_contract.print_payments(value_dt, issuer_curve)

PAYMENT_dt      YEAR_FRAC      FLOW           DF       SURV_PROB      NPV
    20-DEC-2019   0.252778     12638.89     0.998620     0.998658     12604.51
    20-MAR-2020   0.252778     12638.89     0.993728     0.994460     12490.03
    22-JUN-2020   0.261111     13055.56     0.988864     0.990141     12782.88
    21-SEP-2020   0.252778     12638.89     0.984127     0.985978     12263.86
    21-DEC-2020   0.252778     12638.89     0.979968     0.981833     12160.69
    22-MAR-2021   0.252778     12638.89     0.977072     0.977711     12073.86
    21-JUN-2021   0.252778     12638.89     0.974185     0.973608     11987.65
    20-SEP-2021   0.252778     12638.89     0.971306     0.969521     11902.06
    20-DEC-2021   0.252778     12638.89     0.968270     0.965451     11815.05
    21-MAR-2022   0.252778     12638.89     0.964873     0.961400     11724.20
    20-JUN-2022   0.252778     12638.89     0.961488     0.957366     11634.04
    20-SEP-2022   0.255556     12777.78     0.958078     